In [111]:
import pandas as pd
import pandas
import seaborn as sns
import pyodbc
from gensim import corpora, models, similarities
import nltk
import re
import random
import binascii
import logging, gensim, bz2
from nltk.tag import pos_tag
import logging
import re
import tempfile
import tarfile, zipfile
from gensim.utils import tokenize
from collections import defaultdict
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import vertica_python

tenant_id = 289
unuseful_list = 'for a of the and to in is very i my on it was as he she they us this hello hi over';
#search_comment = "";

frequency = defaultdict(int)

conn_info = {'host': 'vertica04.reputation.ec2', 
             'port': 5433, 
             'user': 'rep_engine_user', 
             'password': 'r3p_3ng1n3', 
             'database': 'rwarehouse'}

cnxn = vertica_python.connect(**conn_info)

%matplotlib inline
sample_size = 10000

In [128]:
query = """
SELECT
    tenants.name,
    locations.tenant_id,
    ratings.comment
FROM
    r4e_mongo.mongo_repbiz_location locations,
    r4e_mongo.mongo_repbiz_location_url urls,
    r4e_mongo.mongo_repbiz_rating ratings,
    r4e_mongo.mongo_repbiz_tenant_configurations tenants 
WHERE
    urls.id = locations.id 
    AND ratings.url = urls.url 
    AND locations.tenant_id = tenants.id
    AND (ratings.comment IS NOT NULL OR ratings.comment <> '.' OR ratings.comment <> ' ')
    AND locations.tenant_id = 289
    AND (ratings.source <> '%CARS%' OR ratings.source <> '%SURVEY%' OR ratings.source <> '%KIOSK%')
    AND tenants.active = 'T'
limit 50000
"""

result = pd.read_sql(query, cnxn)

#result

In [113]:
comments = result['comment'].values.tolist();
stoplist = set(unuseful_list.split());

filtered_comment = [[word for word in re.split(r'[;,.!*?~+\r\n\s\' ]\s*', comment.lower()) if word not in stoplist]
      for comment in comments]


for indi_comment in filtered_comment:
    #print(indi_comment)
    #delete all the very short(unuseful) comments
    if len(indi_comment) < 3:
        filtered_comment.remove(indi_comment);
        
for indi_comment in filtered_comment:
    # remove words that appear only once
    for word in indi_comment:
        #pprint(indi_comment)
        frequency[word] +=1;


        #dictionary = corpora.Dictionary.load(result)

In [114]:
from pprint import pprint
pprint(frequency)
filtered_comment = [[token for token in indi_comment if frequency[token] > 1]
        for indi_comment in filtered_comment]

defaultdict(<type 'int'>, {u'': 14671, u'raining': 7, u'aidee': 3, u'courtesous': 1, u'\xa0kyle': 1, u'service-(only': 1, u'foul': 2, u'pony': 1, u'four': 57, u'woods': 2, u'misspelled)': 1, u'hanging': 13, u'woody': 127, u'cyprus': 1, u'gt350': 1, u'payoff': 4, u'increase': 15, u'"guided"': 1, u'eligible': 2, u'sprague': 1, u'star)': 1, u'unanswered': 6, u'fingernails': 1, u'wholeheartedly': 4, u'lori': 8, u'packers': 1, u'sputter': 1, u'lord': 1, u'immature': 1, u'thomasville': 1, u'\xa0our': 4, u'dela': 2, u'"chrysler': 1, u'kannapolis': 2, u'$6500': 2, u'stright': 2, u'retail': 6, u'\xa0since': 2, u'nights': 3, u'beaudoin': 2, u'dell': 3, u'fdcj': 1, u'custom/lifted': 1, u'leisurely': 1, u'spheel': 1, u'malfunctioned': 2, u'"1"': 1, u'bringing': 41, u'theyactually': 1, u'grueling': 3, u'purcahse': 1, u'sturgis': 3, u'wednesday': 33, u'piling': 2, u'multi-point': 4, u'persisted': 2, u'satcher': 6, u'vehical': 10, u'stars)': 2, u'wavered': 1, u'stereotypical': 5, u'commented': 3, u'\

In [115]:
filtered_comment

[[u'staffs', u'were', u'great'],
 [u'excellent', u'service', u'department'],
 [u'thought',
  u'dealership',
  u'pleasant',
  u'deal',
  u'with',
  u'asked',
  u'focus',
  u'se',
  u'hatchback',
  u'never',
  u'felt',
  u'like',
  u'anything',
  u'getting',
  u'pushed',
  u'me',
  u'haven',
  u't',
  u'dealt',
  u'with',
  u'any',
  u'other',
  u'dealerships',
  u'so',
  u'have',
  u'nothing',
  u'service',
  u'against',
  u'but',
  u'staff',
  u'friendly',
  u'even',
  u'picked',
  u'me',
  u'up',
  u'day',
  u'going',
  u'get',
  u'car',
  u'so',
  u'didn',
  u't',
  u'have',
  u'worry',
  u'about',
  u'transportation',
  u''],
 [u'have',
  u'had',
  u'2001',
  u'lincoln',
  u'town',
  u'car',
  u'since',
  u'new',
  u'have',
  u'been',
  u'bringing',
  u'ford',
  u'since',
  u'2005',
  u'its',
  u'service',
  u'with',
  u'great',
  u'satisfaction',
  u'sept',
  u'9th',
  u'we',
  u'traded',
  u'our',
  u'bought',
  u'2016',
  u'ford',
  u'escape',
  u'titanium',
  u'from',
  u'your',

In [116]:
dictionary = corpora.Dictionary(filtered_comment)
dictionary.save('ford.dict')

corpus = [dictionary.doc2bow(indi_comment) for indi_comment in filtered_comment]
corpora.MmCorpus.serialize('/tmp/ford.mm', corpus)


In [117]:
print(dictionary.token2id)

{u'': 6, u'raining': 3125, u'aidee': 9211, u'foul': 5438, u'four': 1112, u'woods': 8822, u'hanging': 3072, u'woody': 3159, u'payoff': 5607, u'increase': 845, u'eligible': 8013, u'unanswered': 1757, u'refunding': 8862, u'lori': 2851, u'\xa0our': 6184, u'dela': 8761, u'kannapolis': 3337, u'stright': 5250, u'retail': 7824, u'\xa0since': 7847, u'regional': 8871, u'dell': 4765, u'yellow': 6364, u'malfunctioned': 7969, u'writing)': 8783, u'bringing': 101, u'grueling': 3315, u'sturgis': 7170, u'wednesday': 2406, u'piling': 7080, u'multi-point': 4129, u'persisted': 7029, u'satcher': 8688, u'vehical': 5464, u'stars)': 2262, u'stereotypical': 4892, u'commented': 1864, u'sawgrass': 8368, u'specially': 1138, u'gaskets': 4553, u'corvette': 3580, u'miller': 2668, u'preface': 5691, u'scrapes': 7782, u'tires': 450, u'scheming': 3328, u'second': 1700, u'currie': 3338, u'sailed': 8777, u'cheverolet': 9073, u'errors': 2059, u'forgetting': 8344, u'fingers': 4410, u'(how': 8618, u'\xa0thanks': 6220, u'cont

In [118]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(4, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 2),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 2),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 2),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1)],
 [(0, 1),
  (4, 1),
  (6, 1),
  (12, 2),
  (26, 1),
  (28, 2),
  (34, 1),
  (37, 2),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 4),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 2),
  (62, 4),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 2),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 3),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 

In [119]:
lsi = models.LsiModel(corpus, id2word = dictionary, num_topics = 2)

In [120]:
#doc = corpus[1]
#doc = dictionary.doc2bow(search_comment.lower().split());

comment_index = random.randrange(0, len(filtered_comment));
search_comment = filtered_comment[comment_index];
doc = dictionary.doc2bow(search_comment);
#search_list = ["pushy", "hassle", "pressured", "high pressure", ""]
#comment_index = random.randrange(0, len());
print(doc)
# lsi = latent semantic indexing
vec_lsi = lsi[doc]
#type(doc)
pprint(vec_lsi)

[(0, 1), (6, 2), (8, 1), (26, 1), (28, 2), (43, 1), (96, 1), (104, 1), (165, 1), (297, 2), (356, 1), (367, 2), (376, 1), (378, 1), (397, 1), (412, 1), (498, 2), (569, 1), (691, 1), (729, 1), (915, 1), (916, 1), (1394, 1), (1417, 1), (1726, 1), (1813, 1), (2021, 1), (2134, 1), (3699, 1)]
[(0, 2.1923900496440352), (1, -1.2463637767463698)]


In [121]:
index = similarities.MatrixSimilarity(lsi[corpus])

In [122]:
type(index)

gensim.similarities.docsim.MatrixSimilarity

In [123]:
#perform similarity query against the corpus
sims = index[vec_lsi]
#pprint(doc[1])
#pprint(dictionary.get(522))


In [124]:
print(list(enumerate(sims)))
#check because there are way too many similar ones if count ~1 are very similar

[(0, 0.71592146), (1, 0.78324974), (2, 0.79198998), (3, 0.70283759), (4, 0.84189063), (5, 0.75687283), (6, 0.98045343), (7, 0.99966347), (8, 0.84622657), (9, 0.76145536), (10, 0.99802846), (11, 0.99293971), (12, 0.58849001), (13, 0.99293804), (14, 0.90908593), (15, 0.64441013), (16, 0.93618894), (17, 0.85244501), (18, 0.94740802), (19, 0.61683649), (20, 0.93015534), (21, 0.79196149), (22, 0.92406595), (23, 0.83996511), (24, 0.9935683), (25, 0.80575854), (26, 0.87890935), (27, 0.91424996), (28, 0.68998158), (29, 0.93461996), (30, 0.89660192), (31, 0.69526649), (32, 0.55993134), (33, 0.92469043), (34, 0.74080795), (35, 0.99970973), (36, 0.93280619), (37, 0.7034905), (38, 0.97872853), (39, 0.8740567), (40, 0.98471618), (41, 0.68155634), (42, 0.96163946), (43, 0.96047032), (44, 0.7674455), (45, 0.97871411), (46, 0.88161886), (47, 0.87262166), (48, 0.65640861), (49, 0.79812551), (50, 0.87379587), (51, 0.97941339), (52, 0.80621725), (53, 0.94320095), (54, 0.96300894), (55, 0.86235499), (56, 

In [125]:
# sort by descending order
sims = sorted(enumerate(sims), key = lambda item: -item[1])
print(sims)

[(5964, 1.0), (8994, 1.0), (2185, 0.99999994), (8960, 0.99999988), (1937, 0.99999982), (2980, 0.99999976), (6346, 0.99999976), (1876, 0.99999958), (6586, 0.9999994), (3633, 0.99999917), (5631, 0.99999911), (4142, 0.99999905), (8315, 0.99999905), (6021, 0.99999881), (5114, 0.99999851), (2950, 0.99999809), (2030, 0.99999791), (8573, 0.99999768), (6044, 0.99999684), (1659, 0.9999963), (804, 0.99999571), (7856, 0.99999511), (5291, 0.99999493), (4379, 0.99999356), (7691, 0.99999356), (2591, 0.99999279), (9226, 0.99999261), (2953, 0.9999913), (160, 0.99999088), (2944, 0.99999017), (7007, 0.99998981), (7697, 0.99998981), (8338, 0.99998951), (6608, 0.99998945), (4140, 0.99998933), (5669, 0.99998933), (1928, 0.99998891), (4204, 0.99998844), (8433, 0.99998844), (5320, 0.99998802), (6829, 0.99998802), (4994, 0.99998677), (5710, 0.99998671), (6029, 0.99998665), (6779, 0.99998665), (2860, 0.99998647), (5924, 0.99998617), (781, 0.99998581), (4854, 0.99998552), (3250, 0.99998468), (8372, 0.99998468),

In [126]:
filtered_comment[3354]
print(comment_index)

8994


In [127]:
print("Initial comment: " + str(filtered_comment[comment_index]) + "\n**********************************\n");

index = 0;
while index < 20:
    print("Comment: "+ str(filtered_comment[sims[index][0]]));
    print("Similarity: " + str.format('{0:.15f}', sims[index][1]) + "\n");
    #print(sims[index]);
    index += 1;

Initial comment: [u'brandon', u'extremely', u'knowledgable', u'product', u'helped', u'work', u'great', u'deal', u'finance', u'manager', u'sales', u'manager', u'though', u'don', u't', u'remember', u'their', u'names', u'extremely', u'nice', u'work', u'with', u'pleasant', u'ford', u'exceeded', u'expectations', u'look', u'forward', u'working', u'with', u'them', u'again', u'', u'']
**********************************

Comment: [u'would', u'recommend', u'spikes', u'ford', u'everyone', u'valley', u'have', u'best', u'when', u'comes', u'customer', u'service', u'customer', u'care', u'customer', u'concerns', u'needs', u'are', u'customer', u'oriented', u'', u'', u'better', u'than', u'another', u'ford', u'dealership', u'that', u'could', u'mention', u'real', u'fast', u'that', u'refused', u'take', u'care', u'brand', u'new', u'car', u'that', u'had', u'purchased', u'less', u'than', u'month', u'from', u'', u'', u'']
Similarity: 1.000000000000000

Comment: [u'brandon', u'extremely', u'knowledgable', u'pro